In [22]:
import os
from river import datasets
from river import metrics
from river import tree
from river import ensemble
from river import evaluate
from river import compose
from river import naive_bayes
from river import anomaly
from river import compose
from river import datasets
from river import metrics
from river import preprocessing
from confluent_kafka import Producer,Consumer
import certifi
import time
import json
import pandas

In [23]:
classifiers=[]
classifier='HoeffdingAdaptiveTreeClassifier'
classifiers.append(classifier)
classifier='SRPClassifierHAT'
classifiers.append(classifier)
classifier='SRPClassifierNaiveBayes'
classifiers.append(classifier)
classifier='AdaptiveRandomForestClassifier'
classifiers.append(classifier)
#classifier='HalfSpaceTrees'
#classifiers.append(classifier)
model_artifact = None

In [24]:
max_size=1000
dataset = datasets.MaliciousURL()
d_train=[]
d_predict=[]
d_total=[]
for classifier in classifiers:
    data = dataset.take(max_size)
    if classifier=='HoeffdingAdaptiveTreeClassifier':
            model_artifact = tree.HoeffdingAdaptiveTreeClassifier(grace_period=100,  delta=1e-5, leaf_prediction='nb', nb_threshold=10,seed=0)
    elif classifier=='SRPClassifierHAT':
            model_artifact =  ensemble.SRPClassifier(
                                                   model=tree.HoeffdingAdaptiveTreeClassifier(grace_period=100,  delta=1e-5, leaf_prediction='nb', nb_threshold=10, seed=0), seed=42,
                                                )
    elif classifier=='SRPClassifierNaiveBayes':
            model_artifact = ensemble.SRPClassifier(
                                                 model=naive_bayes.BernoulliNB(alpha=1), seed=42,
                                              )
    elif classifier=='AdaptiveRandomForestClassifier':
            model_artifact = ensemble.AdaptiveRandomForestClassifier(leaf_prediction="mc")

#    elif classifier=='HalfSpaceTrees':
#            model_artifact = compose.Pipeline(preprocessing.MinMaxScaler(),anomaly.HalfSpaceTrees(seed=42))
    print(model_artifact)
    cnt = 0
    training_st_ts = time.time()
    for f, y in data:
        cnt = cnt + 1
        model_artifact = model_artifact.learn_one(f,y)
    training_end_ts = time.time()
    total_training_duration = training_end_ts-training_st_ts
    avg_duration = (total_training_duration/max_size)     
    d_train.append({'Classifier':classifier,
              'Expected Time(ms) Per Record': avg_duration * 1000,
              'Expected Time Million Records (s)':avg_duration * 1000000,
              'Expected Time Million Records (mins)': avg_duration * (1000000/60),
              'Max Training Throughput (sub-second-response) ': round(1/avg_duration)})
    
    data = dataset.take(max_size)
    prediction_st_ts = time.time()
    for f, y in data:
        cnt = cnt + 1
        if classifier=='HalfSpaceTrees':
            model_artifact.learn_one(f)
        else:
            model_artifact.predict_one(f)
    prediction_end_ts = time.time()
    total_prediction_duration = prediction_end_ts-prediction_st_ts
    avg_duration = (total_prediction_duration/max_size)     
    d_predict.append({'Classifier':classifier,
              'Expected Time(ms) Per Record': avg_duration * 1000,
              'Expected Time Million Records (s)':avg_duration * 1000000,
              'Expected Time Million Records (mins)': avg_duration * (1000000/60),
              'Max Prediction Throughput (sub-second-response) ': round(1/avg_duration)})

    total_duration = total_training_duration + total_prediction_duration
    avg_duration = (total_duration/max_size)     
    d_total.append({'Classifier':classifier,
              'Expected Time(ms) Per Record': avg_duration * 1000,
              'Expected Time Million Records (s)':avg_duration * 1000000,
              'Expected Time Million Records (mins)': avg_duration * (1000000/60),
              'Max (Training+Prediction) Throughput (sub-second-response) ': round(1/avg_duration)})



HoeffdingAdaptiveTreeClassifier
SRPClassifier(HoeffdingAdaptiveTreeClassifier)
SRPClassifier(BernoulliNB)
AdaptiveRandomForestClassifier


In [25]:
print('Training')
df_train = pandas.DataFrame(data=d_train)
display(df_train)

print('Prediction')
df_predict = pandas.DataFrame(data=d_predict)
display(df_predict)

print('Total')
df_total = pandas.DataFrame(data=d_total)
display(df_total)

Training


,Classifier,Expected Time(ms) Per Record,Expected Time Million Records (s),Expected Time Million Records (mins),Max Training Throughput (sub-second-response)
0,HoeffdingAdaptiveTreeClassifier,1.844334,1844.333887,30.738898,542
1,SRPClassifierHAT,28.673409,28673.409462,477.890158,35
2,SRPClassifierNaiveBayes,4.351867,4351.867199,72.531120,230
3,AdaptiveRandomForestClassifier,0.934213,934.213161,15.570219,1070


Prediction


,Classifier,Expected Time(ms) Per Record,Expected Time Million Records (s),Expected Time Million Records (mins),Max Prediction Throughput (sub-second-response)
0,HoeffdingAdaptiveTreeClassifier,0.554865,554.865122,9.247752,1802
1,SRPClassifierHAT,1.242568,1242.568254,20.709471,805
2,SRPClassifierNaiveBayes,1.139312,1139.311552,18.988526,878
3,AdaptiveRandomForestClassifier,0.169602,169.602394,2.826707,5896


Total


,Classifier,Expected Time(ms) Per Record,Expected Time Million Records (s),Expected Time Million Records (mins),Max (Training+Prediction) Throughput (sub-second-response)
0,HoeffdingAdaptiveTreeClassifier,2.399199,2399.199009,39.986650,417
1,SRPClassifierHAT,29.915978,29915.977716,498.599629,33
2,SRPClassifierNaiveBayes,5.491179,5491.178751,91.519646,182
3,AdaptiveRandomForestClassifier,1.103816,1103.815556,18.396926,906


## Results for Maximum Sustainable Throughtput Calculations 
Maximum sustainable throughput is the number of records we can train/predict in a single thread in 1 second